### Collab

#### git clone

In [ ]:
#!git clone https://github.com/<your name>/<your project>.git
#!git clone https://github.com/leaguilar/AIknowthatfeel/ colab
#pip install -r requirements.txt
#import sys
#sys.path.append('colab')

### Importamos las librerias y el dataset

In [ ]:
import numpy as np
import os
import shutil
import datetime

In [ ]:
from keras.models import Sequential
from keras.preprocessing import sequence
from keras.layers import Embedding, LSTM, Dense
from keras.callbacks import TensorBoard
from keras.preprocessing.text import one_hot
import pandas as pd

In [ ]:
from libs.workshop_libs.tensorboard import TrainValTensorBoard

### Importar el dataset

In [ ]:
#Ugly Example preprocessing
pad_char=0
start_char=1
oov_char=2
index_from=3

with open('models/wordDictionary.pkl', 'rb') as handle:
    wordDict = pickle.load(handle)

def encodeList(text_list,vocab_size):
    encoded=[start_char]
    for w in text_list:
        val=wordDict.get(w,oov_char-index_from)+index_from
        if val>=vocab_size:
            val=vocab_size-1
        encoded.append(val)
    #print(encoded)
    return encoded
       
def encodeData(data,vocab_size = 5000):
    data_out=[]
    for index, row in data.iterrows():
        text=(row.iloc[0]+' ### '+row.iloc[1]+' ### '+row.iloc[2])
        text_list=text.split()
        encoded=encodeList(text_list,vocab_size)
        data_out.append(encoded)
    return np.array(data_out)
def encodedLabels(data):
    data_out=[]
    for index, row in data.iterrows():
        if row.iloc[0] == 'others':
            data_out.append([1,0,0,0])
        elif row.iloc[0] == 'happy':
            data_out.append([0,1,0,0])
        elif row.iloc[0] == 'angry':
            data_out.append([0,0,1,0])
        elif row.iloc[0] == 'sad':
            data_out.append([0,0,0,1])
        else:
            raise Exception()
    return np.array(data_out).astype(int)
def prepare_data(x_data,max_conv_length):
    x_data = sequence.pad_sequences(x_data, maxlen=max_conv_length)
    return x_data
def loadAndPrepareData(fname,vocab_size=5000,max_conv_length=50):
    data = pd.read_csv(fname,sep='	',header=None)
    x_data_raw=data.filter([1,2,3], axis=1).copy(deep=True)
    y_data_raw=data.filter([4], axis=1).copy(deep=True)  
    x_data = encodeData(x_data_raw,vocab_size)
    y_data = encodedLabels(y_data_raw)
    x_data = prepare_data(x_data,max_conv_length)                       
    return x_data,y_data

In [ ]:
vocab_size=5000
max_conv_length=50

In [ ]:
x_train,y_train=loadAndPrepareData('data_train/train.txt',vocab_size,max_conv_length)
x_dev,y_dev=loadAndPrepareData('data_dev/dev.txt',vocab_size,max_conv_length)

In [ ]:
len(x_train[0])

In [ ]:
x_train[0][:10]

In [ ]:
len(x_train)

In [ ]:
#QUITAR
max_data=10000
x_train=x_train[0:max_data]
y_train=y_train[0:max_data]
x_dev=x_dev[0:max_data]
y_dev=y_dev[0:max_data]

### Revisemos el dataset

In [ ]:
## Cargamos tensorboard

In [ ]:
%load_ext tensorboard
#%reload_ext tensorboard

In [ ]:
#!kill 4430

In [ ]:
#!ps aux | grep -i tb

In [ ]:
logs_base_dir = "./logs"
print(shutil.rmtree(logs_base_dir))

In [ ]:
os.makedirs(logs_base_dir, exist_ok=True)
%tensorboard --logdir {logs_base_dir}

In [ ]:
### Uniformizemos los datos, truncados y padding

In [ ]:
def create_model(top_words,max_review_length):
    embedding_vector_length = 32
    model = Sequential()
    model.add(Embedding(top_words, embedding_vector_length, input_length=max_review_length))
    model.add(LSTM(100))
    model.add(Dense(4, activation='sigmoid'))
    print(model.summary())
    return model

In [ ]:
def train_model(model,x_train,y_train,x_dev,y_dev):
        logdir = os.path.join(logs_base_dir, datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
        model.compile(loss='categorical_crossentropy',
                      optimizer='adam',
                      metrics=['accuracy']
                     )
        model.fit(x_train,
                  y_train,
                  batch_size=64, #64,
                  validation_data=(x_dev, y_dev),
                  nb_epoch=10,
                  callbacks=[TrainValTensorBoard(logdir, 
                                                 histogram_freq=1,
                                                 write_graph=True
                                                )]
                 )

In [ ]:
print(x_train.shape)
print(x_dev.shape)

In [ ]:
model=create_model(vocab_size,max_conv_length)

In [ ]:
train_model(model,x_train,y_train,x_dev,y_dev)

### Evaluamos el modelo

In [ ]:
scores = model.evaluate(x_dev, y_dev, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

### Guardamos el modelo

In [ ]:
outpath='models/'
os.makedirs(outpath, exist_ok=True)

In [ ]:
model.save(outpath+"model.h5")

In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open(outpath+"model.json", "w") as json_file:
    json_file.write(model_json)

In [ ]:
from keras.models import load_model

In [ ]:
model2=load_model("models/model.h5")

In [ ]:
scores = model2.evaluate(x_dev, y_dev, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
print(model.metrics_names)

### ADDENDUM: One hot encoder

In [ ]:
from keras.preprocessing.text import one_hot

In [ ]:
texto=["Esta es una frase","Otra frase mas larga", "todo esto viene de nuestro dataset de interes"]

In [ ]:
vocab_size = 50
palabras_vectorizadas = [one_hot(d, vocab_size) for d in texto]
print(palabras_vectorizadas)

### Problemas con numpy nuevos y keras

In [ ]:
# save np.load
np_load_old = np.load

# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

# call load_data with allow_pickle implicitly set to true
#(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

In [ ]:
# restore np.load for future normal usage
np.load = np_load_old

In [ ]:
### Otra solucion

In [ ]:
#!pip uninstall numpy

In [ ]:
#!pip install numpy==1.16.1